In [1]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

There's a new jupysql version available (0.11.1), you're running 0.10.10. To upgrade: pip install jupysql --upgrade
Deploy Panel apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


Loading configurations from /workspaces/fundamentals-of-data-transformation-4554074/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaylimit,10
displaycon,False


,Count
0,224


Now we get to have some fun! **Window** functions works by breaking up relations into _independent_ partitions, ordering those partitions and then computing a new column for each row. 

<div align="center">
<img src = "https://bs-uploads.toptal.io/blackfish-uploads/uploaded_file/file/22142/image-1561028553443-05eed70976f64c363db78d603cba17d6.png" width=60% />
</div>

If this sounds complex, it might be at first, but we'll break it down for you. If this sounds computationally intensive, it is... So we'll have to be careful when windowing over large datasets.

When we build a window function, we're basically iterating over a slice of data _relative_ to the other rows around it. This is a powerful pattern and one that makes SQL stand out as an excellent language for querying relational data.

<div align="center">
<img src = "https://duckdb.org/images/blog/windowing/framing.svg" width=30% />
</div>


Here are some sample questions well suited to windows:
- Which park has the most campsites?
- What's the second largest campsite in each park?
- What is the running total of alerts for all parks?
- What's the 7-day moving average of alerts?

Window functions are made up of three parts— the _function_, a _partition_, and an _order_. 

- First, we describe what we'd like to return, maybe the `MAX` value or just the `FIRST` value.
- Next, we describe the partition. A partition is just a way of describing the "chunks" we want to query our data by. It's almost like a `GROUP`, for example if we parition by `park_name`, we'll get the `MAX` _for each_ park name.
- Then, we define an order. This isn't necessary in queries like `MAX`, but for a `FIRST` query, order is essential.
- Finally, we can define a _frame_, which tells us which rows to consider. We'll talk more about frames later.
  

Below is an example:

In [12]:
%%sql

SELECT *
FROM nps_public_data.parks p
limit 1


,relevanceScore,designation,weatherInfo,addresses,operatingHours,entrancePasses,name,description,directionsUrl,fees,...,activities,url,longitude,id,images,directionsInfo,fullName,parkCode,latLong,latitude
0,1,National Memorial,http://forecast.weather.gov/MapClick.php?CityN...,"[{'type': 'Physical', 'line2': '', 'line1': '1...","[{'name': 'Hours of Operation', 'standardHours...",[],Federal Hall,"Here on Wall Street, George Washington took th...",http://www.nps.gov/feha/planyourvisit/directio...,[],...,"[{'name': 'Arts and Culture', 'id': '09DF0950-...",https://www.nps.gov/feha/index.htm,-74.010256,2337D255-2D32-4997-957A-D461EEA03AF8,[{'url': 'https://www.nps.gov/common/uploads/s...,The main entrance of Federal Hall is located a...,Federal Hall National Memorial,feha,"lat:40.70731192, long:-74.01025636",40.707312


In [4]:
%%sql
SELECT
    DISTINCT p.fullname as park_name,

    -- For each park, which campground has the maximum number of campsites?
    MAX(c.numberofsitesfirstcomefirstserve) OVER (PARTITION BY park_name) as max_num_fcfc,
    MAX(c.numberofsitesreservable) OVER (PARTITION BY park_name) as max_num_reserve,
    MAX(c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable) OVER (PARTITION BY park_name) as max_num_campsites,

    -- For each park, which _campsite_ has the maximum number of campsites?
    FIRST(c.name) OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve DESC) as max_num_fcfs_site,
    FIRST(c.name) OVER (PARTITION BY park_name ORDER BY c.numberofsitesreservable DESC) as max_num_reserve_site,
    FIRST(c.name) OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as max_num_campsites_site
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation = 'National Park'
ORDER BY max_num_campsites DESC
LIMIT 5;

,park_name,max_num_fcfc,max_num_reserve,max_num_campsites,max_num_fcfs_site,max_num_reserve_site,max_num_campsites_site
0,Mesa Verde National Park,267,267,534,Morefield Campground,Morefield Campground,Morefield Campground
1,Yellowstone National Park,0,432,432,Bridge Bay Campground,Bridge Bay Campground,Bridge Bay Campground
2,Grand Teton National Park,10,347,347,Jenny Lake Campground,Colter Bay Campground,Colter Bay Campground
3,Grand Canyon National Park,15,300,315,Mather Campground - South Rim,Mather Campground - South Rim,Mather Campground - South Rim
4,Yosemite National Park,156,235,304,Tuolumne Meadows Campground,Upper Pines Campground,Tuolumne Meadows Campground


`LEAD`, `LAG`, & `NTH_VALUE` are similar functions. As the names suggest:
- `LEAD` returns the _next_ value in a particular partition.
- `LAG` returns the _previous_ value in a particular partition.
- `NTH_VALUE` returns a value the index you provide.

These functions are interchangeable, for example lead and lag often take parameters that specify how many rows to skip, so `LEAD(name, 2) =  LAG(name, -2) = NTH_VALUE(name, 2)`.

That can be confusing, so we recommend sticking with one.

In [14]:
%%sql
SELECT
    p.fullname as park_name,
    c.name as campground_name,
    c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as num_sites,
    -- LAG & LEAD
    -- Get the previous campsite ordered by number of sites.
    LAG(c.name) OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as next_campsite,
    -- Get the next campsite ordered by number of sites.
    LEAD(c.name) OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as previous_campsite,
    -- Get the next next campsite ordered by number of sites.
    LEAD(c.name, 2) OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as previous_previous_campsite,
    -- The default values for frames are from UNBOUNDED PRECEDING to CURRENT ROW
    NTH_VALUE(c.name, 2) OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as second_campsite,
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation = 'National Park'
WHERE p.fullname = 'Death Valley National Park'
ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable
LIMIT 5

,park_name,campground_name,num_sites,next_campsite,previous_campsite,previous_previous_campsite,second_campsite
0,Death Valley National Park,Eureka Dunes Campground (Primitive),5,Thorndike Campground (Primitive),None,None,Stovepipe Wells Campground
1,Death Valley National Park,Thorndike Campground (Primitive),6,Mahogany Flat Campground (Primitive),Eureka Dunes Campground (Primitive),None,Stovepipe Wells Campground
2,Death Valley National Park,Mahogany Flat Campground (Primitive),9,Homestake Campground (Primitive),Thorndike Campground (Primitive),Eureka Dunes Campground (Primitive),Stovepipe Wells Campground
3,Death Valley National Park,Saline Valley Campground (Primitive),10,Wildrose Campground,Emigrant Campground,Homestake Campground (Primitive),Stovepipe Wells Campground
4,Death Valley National Park,Emigrant Campground,10,Saline Valley Campground (Primitive),Homestake Campground (Primitive),Mahogany Flat Campground (Primitive),Stovepipe Wells Campground


`RANK` functions can be particularly useful. There are three main functions, `ROW_NUMBER`, `RANK`, and `DENSE_RANK`. They're slightly different but very similar:

- `ROW_NUMBER` monotonically increases, regardless of ties.
- `RANK` and `DENSE_RANK`— ties get the same number, e.g. 1, 2, 3, 4, 4, 4.
- `DENSE_RANK` continues iterating after a tie, `RANK` does not, e.g. 1, 2, 3, 3, 5 vs. 1, 2, 3, 3, 4.

In [15]:
%%sql
SELECT
    p.fullname as park_name,
    c.name as campground_name,

    -- For each park, which campground has the maximum number of campsites?
    c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as num_campsites,
    -- RANK, ROW_NUMBER, DENSE_RANK     
    RANK() OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as park_campsites_rank,
    ROW_NUMBER() OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as campsites_row_num,
    DENSE_RANK() OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as campsites_dense_rank,
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation = 'National Park'
WHERE p.fullname = 'Death Valley National Park'
ORDER BY park_name, park_campsites_rank ASC
LIMIT 12;

,park_name,campground_name,num_campsites,park_campsites_rank,campsites_row_num,campsites_dense_rank
0,Death Valley National Park,Sunset Campground,230,1,1,1
1,Death Valley National Park,Stovepipe Wells Campground,190,2,2,2
2,Death Valley National Park,Furnace Creek Campground,136,3,3,3
3,Death Valley National Park,Texas Springs Campground,92,4,4,4
4,Death Valley National Park,Mesquite Spring Campground,40,5,5,5
5,Death Valley National Park,Wildrose Campground,23,6,6,6
6,Death Valley National Park,Saline Valley Campground (Primitive),10,7,7,7
7,Death Valley National Park,Emigrant Campground,10,7,8,7
8,Death Valley National Park,Homestake Campground (Primitive),10,7,9,7
9,Death Valley National Park,Mahogany Flat Campground (Primitive),9,10,10,8


Let's talk about _rolling averages_. Rolling averages are a concept that are _much_ easier grasped in SQL thanks to the row-based layout and language details.

Frames can be very useful in rolling averages.

In [2]:
%%sql
SELECT
    a.lastindexeddate::DATE as dt,
    COUNT(*) as num_alerts,
FROM nps_public_data.alerts a
LEFT JOIN nps_public_data.parks p
    USING(parkcode)
WHERE dt BETWEEN '2023-01-01' AND '2023-12-31'
GROUP BY 1
ORDER BY dt ASC
LIMIT 10

,dt,num_alerts
0,2023-01-04,2
1,2023-01-07,1
2,2023-01-09,1
3,2023-01-15,1
4,2023-01-17,1
5,2023-01-20,1
6,2023-02-03,1
7,2023-02-08,2
8,2023-02-10,1
9,2023-02-16,1


In [ ]:
%%sql
SELECT
    a.lastindexeddate::DATE as dt,
    COUNT(*) as num_alerts,
    SUM(COUNT(*)) OVER (ORDER BY dt) as cum_num_alerts
FROM nps_public_data.alerts a
INNER JOIN nps_public_data.parks p
    USING(parkcode)
WHERE dt BETWEEN '2023-06-01' AND '2024-7-31'
GROUP BY 1
ORDER BY dt ASC
LIMIT 10

But our dates are not consecutive (not every day has an alert!), so an average is difficult. We can generate a series to get around this.

In [4]:
%%sql
  SELECT
     generate_series as dt
  FROM
     generate_series(timestamp '2019-01-01', timestamp '2025-01-01', interval '1 day')


,dt
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05
...,...
2188,2024-12-28
2189,2024-12-29
2190,2024-12-30
2191,2024-12-31


In [5]:
%%sql
WITH date_arr AS (
  -- We'll discuss this more in the lesson on data generation!
  SELECT
     generate_series as dt
  FROM
     generate_series(timestamp '2019-01-01', timestamp '2025-01-01', interval '1 day')
)
SELECT
   da.dt,
    COUNT(a.id) as num_alerts,
    SUM(COUNT(a.id)) OVER (ORDER BY da.dt ASC) as cum_num_alerts
FROM date_arr da
LEFT JOIN nps_public_data.alerts a
    ON da.dt::DATE = a.lastindexeddate::DATE
LEFT JOIN nps_public_data.parks p
    USING(parkcode)
WHERE da.dt BETWEEN '2023-06-01' AND '2024-7-31'
GROUP BY 1
ORDER BY da.dt ASC
LIMIT 15;

,dt,num_alerts,cum_num_alerts
0,2023-06-01,0,0.0
1,2023-06-02,0,0.0
2,2023-06-03,0,0.0
3,2023-06-04,0,0.0
4,2023-06-05,1,1.0
5,2023-06-06,2,3.0
6,2023-06-07,0,3.0
7,2023-06-08,1,4.0
8,2023-06-09,0,4.0
9,2023-06-10,0,4.0


And now get our rolling counts + averages!

In [ ]:
%%sql
WITH date_arr AS (
  -- We'll discuss this more in the lesson on data generation!
  SELECT
     generate_series as dt
  FROM
     generate_series(timestamp '2019-01-01', timestamp '2025-01-01', interval '1 day')
)
SELECT
   da.dt,
    COUNT(a.id) as num_alerts,
    SUM(COUNT(a.id)) OVER (ORDER BY da.dt ASC) as cum_num_alerts,
    -- Note the definition of our "frame"
    SUM(COUNT(a.id)) OVER (ORDER BY da.dt ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as seven_day_rolling,
    ROUND(
        AVG(COUNT(a.id)) OVER (ORDER BY da.dt ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
        , 2) as seven_day_rolling_avg,
FROM date_arr da
LEFT JOIN nps_public_data.alerts a
    ON da.dt::DATE = a.lastindexeddate::DATE
LEFT JOIN nps_public_data.parks p
    USING(parkcode)
WHERE da.dt BETWEEN '2023-06-01' AND '2024-7-31'
GROUP BY 1
ORDER BY da.dt ASC
LIMIT 15;

Note how the _frame_ clause is instructing our query about which rows to aggregate!

So that's the basics of window functions. If you can understand the idea of breaking down the function into _three_ components— the aggregate, the partition, and the frame, you've mastered windows! Yes, this takes quite a bit of time, but once you get it, it totally makes sense!